In [ ]:
#!pip install -r requirements.txt

In [ ]:
import ee
#ee.Authenticate()

In [ ]:
from utils import *



In [ ]:


ee.Initialize()

In [ ]:
def getFeatureRaster(feature, exclusionRaster, targetRaster):
    excls = exclusionRaster.clip(feature)
    targ = targetRaster.clip(feature)

    return targ.multiply(excls)

In [ ]:
def reduceToArea(image):
    areaDict = image.pixelArea().reduceRegion(ee.Reducer.sum(), scale=140, geometry=image.geometry())
    area = ee.Number(areaDict.get('area')).divide(1e6)
    name = image.get("name")
    newFeat = ee.Feature(image.geometry())
    newFeat = newFeat.set({'area':area, 'name':name})
    #newFeat = ee.Feature.setGeometry()
    return newFeat

In [ ]:
# Scenario 1 - Max Exclusions


common = getExclusions()
wind = getExclusions("wind")
solar = getExclusions("solar")


#Scenario 1 - Max Exclusions
MaxWindExclusions = common + wind
MaxSolarExclusions = common + solar

maxExcl = {"wind":MaxWindExclusions, "solar": MaxSolarExclusions}

#Scenario 2 - Med Exclusions: 
MedWindExclusions = common + wind
MedWindExclusions.remove("Peatland")
MedWindExclusions.remove("Areas of Natural Beauty")

MedSolarExclusions = common + solar
MedSolarExclusions.remove("Peatland")
MedSolarExclusions.remove("Areas of Natural Beauty")

medExcl = {"wind":MedWindExclusions, "solar": MedSolarExclusions}

#Scenario 3 - Min Exclusions
MinWindExclusions = common + wind
MinWindExclusions.remove("Peatland")
MinWindExclusions.remove("Areas of Natural Beauty")
MinWindExclusions.remove("England National Parks")
MinWindExclusions.remove("Scotland Scenic Areas")
MinWindExclusions.remove("Wales National Park")

MinSolarExclusions = common + solar
MinSolarExclusions.remove("Peatland")
MinSolarExclusions.remove("Areas of Natural Beauty")
MinSolarExclusions.remove("England National Parks")
MinSolarExclusions.remove("Scotland Scenic Areas")
MinSolarExclusions.remove("Wales National Park")

minExcl = {"wind":MinWindExclusions, "solar": MinSolarExclusions}


paramsDict = {"min":minExcl, "med":medExcl, "max": maxExcl}


def getPowerLocal(geometry, exclusions, power):
    localPower = power.multiply(exclusions).clip(geometry)
    return localPower

exDict = getExclusionsDict()
resultsDict = {}
paramList = list(paramsDict.keys())


ladNameList = load_csv_list("local_authorities_name.csv")[1:]

LADVector= ladsVector()

for idx, exp in enumerate(paramList):
    exclDict = getExclusionsDict()
    windExc = paramsDict[exp]['wind']
    solarExc = paramsDict[exp]['solar']
    print(windExc)
    
    #windExclusionList = [exclDict[x] for x in windExc]
    #for ex in windExclusionList:                      
    
    names = []
    solarAreas = []
    windAreas = []
    
    
    for idx2, geomName in enumerate(ladNameList):
        try:
            areaRast = ee.Image.constant(1)
            areaRast = areaRast.reproject(areaRast.projection().atScale(100)).pixelArea()

            activeFeat = LADVector.filter(f"LAD21NM == '{geomName}'").first()
            activeGeom = activeFeat.geometry()

            areaRast = areaRast.clip(activeGeom)


            def getTotalArea(feat):
                return areaRast.reduceRegion(ee.Reducer.sum(), feat, 100)



            for x in windExc:
                ex = exclDict[x]
                areaRast = areaRast.multiply(ex)
            areaRast = minimumMappingUnit(areaRast, 7)
            windArea = getTotalArea(activeGeom).getInfo()['area']


            areaRast = ee.Image.constant(1)
            areaRast = areaRast.reproject(areaRast.projection().atScale(100)).pixelArea()

            for x in solarExc:
                ex = exclDict[x]
                areaRast = areaRast.multiply(ex)
            areaRast = minimumMappingUnit(areaRast, 7)
            solarArea = getTotalArea(activeGeom).getInfo()['area']
            names.append(geomName)
            solarAreas.append(solarArea)
            windAreas.append(windArea)
            print(geomName, 'wind power', windArea, 'solar power', solarArea)
            print(idx2, 'of', len(ladNameList), 'done')
        except:
            names.append(geomName)
            solarAreas.append('failed')
            windAreas.append('failed')
            print(idx2, 'of', len(ladNameList), 'failed')

    resultsFrame = pd.DataFrame([names, solarAreas, windAreas]).T
    resultsFrame.columns = ['area name', 'solar power area', 'wind powe area']
    resultsFrame.to_csv(f'{exp}.csv')
    resultsDict[exp] = resultsFrame
    
        #print(getTotalArea(LADVector.first().geometry()).getInfo())
    
    
        
    


string = '''
for exp in paramList:
    
    areaRast = ee.Image.constant(1)
    windExc = paramsDict[exp]['wind']
    solarExc = paramsDict[exp]['solar']
    print(solarExc)
    assert windExc != solarExc

    windExclusions = [exDict[x] for x in windExc]
    solarExclusions = [exDict[x] for x in solarExc]

    windExclusions = compute_exclusions(windExclusions)
    solarExclusions = compute_exclusions(solarExclusions)

    featCol = constituenciesVector()



    solarAreaImgCol = ee.ImageCollection(featCol.map(lambda x: getFeatureRaster(x, solarExclusions, areaRast).set("name", ee.String(x.get('pcon19nm'))) ))
    windAreaImgCol = ee.ImageCollection(featCol.map(lambda x: getFeatureRaster(x, windExclusions, areaRast).set("name", ee.String(x.get('pcon19nm'))) ))
    
    #windAreaImgCol = windAreaImgCol.updateMask(solarAreaImgCol.gte(1))

    windAreaReduced = windAreaImgCol.map(reduceToArea)
    windarr = windAreaReduced.aggregate_array('area').getInfo()
    windNames = windAreaReduced.aggregate_array('name').getInfo()
    windPFrame = pd.DataFrame(windarr, index=windNames, columns=["Wind Area (KM^2)"])

    solarAreaReduced = solarAreaImgCol.map(reduceToArea)
    solarr = solarAreaReduced.aggregate_array('area').getInfo()
    solNames = solarAreaReduced.aggregate_array('name').getInfo()
    solPFrame = pd.DataFrame(solarr, index=solNames, columns=["Solar Area (KM^2)"])

    potentialFrame = solPFrame.join(windPFrame)
    assert solarr != windarr
    resultsDict[exp] = potentialFrame
'''

In [ ]:
    resultsFrame = pd.DataFrame([names, solarAreas, windAreas]).T
    resultsFrame.columns = ['area name', 'solar power area', 'wind powe area']


In [ ]:
resultsFrame

In [ ]:
solarExclusions

In [ ]:
break

In [ ]:
solarExcmin = paramsDict["min"]['solar']
solarExcmax = paramsDict["max"]['solar']
solExclMin = [exDict[x] for x in solarExcmin]
solExclMax = [exDict[x] for x in solarExcmax]

In [ ]:
solExclMin == solExclMax

In [ ]:
solarExcmax

In [ ]:
potentialFrame

In [ ]:
constVect.first().get('pcon19nm').getInfo()

In [ ]:
constVect.map(lambda x: getPowerLocal(x, solarExclusions, areaRast)).getInfo()

In [ ]:
solarr == windarr

In [ ]:
solarr 

In [ ]:
areaReduced.first().getInfo()